<a href="https://colab.research.google.com/github/AYA0HASSAN/Pose_detection/blob/Aya_branch/Final_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import scipy.io as sio
import os
import tensorflow as tf

import cv2
import dlib

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

# import mlflow
# from mlflow.tracking import MlflowClient

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from google.colab import drive

In [ ]:
from google.colab import files
import io
uploaded = files.upload()


 
data_df = pd.read_csv(io.BytesIO(uploaded['data.csv']))
data_df.head()

Saving data.csv to data (1).csv


,nose_x,nose_y,chin_x,chin_y,righteye_rc_x,righteye_rc_y,lefteye_lc_x,lefteye_lc_y,mouth_rc_x,mouth_rc_y,mouth_lc_x,mouth_lc_y,pitch,yaw,roll
0,210.793304,282.391510,182.876678,362.098145,244.540939,284.477783,291.340729,207.794174,273.853607,218.519974,248.992859,313.858582,-0.399231,0.018227,0.085676
1,200.270554,278.768677,205.846298,347.868408,212.636292,275.970337,246.227020,219.497284,240.278381,228.604675,211.068298,296.566040,0.470065,1.189533,0.300959
2,207.861694,282.409760,229.155289,349.297302,224.289276,278.439453,244.417877,229.510941,238.263702,236.791260,233.105896,297.185791,-0.184650,0.881137,-0.236852
3,219.280930,292.207214,244.332855,362.732819,242.067657,282.309021,254.078079,205.115799,246.204193,219.895065,254.550903,301.377075,-0.175379,0.299208,-0.373374
4,197.625732,287.040161,254.701157,354.565582,207.167908,280.051086,204.078888,228.311523,208.819092,234.895508,213.983398,300.547394,-0.882169,1.198004,-1.033374


In [ ]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   nose_x         2000 non-null   float64
 1   nose_y         2000 non-null   float64
 2   chin_x         2000 non-null   float64
 3   chin_y         2000 non-null   float64
 4   righteye_rc_x  2000 non-null   float64
 5   righteye_rc_y  2000 non-null   float64
 6   lefteye_lc_x   2000 non-null   float64
 7   lefteye_lc_y   2000 non-null   float64
 8   mouth_rc_x     2000 non-null   float64
 9   mouth_rc_y     2000 non-null   float64
 10  mouth_lc_x     2000 non-null   float64
 11  mouth_lc_y     2000 non-null   float64
 12  pitch          2000 non-null   float64
 13  yaw            2000 non-null   float64
 14  roll           2000 non-null   float64
dtypes: float64(15)
memory usage: 234.5 KB


In [ ]:
features = data_df.drop(['pitch','yaw','roll'],axis = 1)
labels = data_df[['pitch','yaw','roll']]
pitch = data_df['pitch']
yaw = data_df['yaw']
roll = data_df['roll']

## Read Video

In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

--2022-05-17 16:35:55--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  57.3MB/s    in 1.1s    

2022-05-17 16:35:57 (57.3 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
!wget http://arunponnusamy.com/files/mmod_human_face_detector.dat

--2022-05-17 16:36:35--  http://arunponnusamy.com/files/mmod_human_face_detector.dat
Resolving arunponnusamy.com (arunponnusamy.com)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to arunponnusamy.com (arunponnusamy.com)|185.199.108.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.arunponnusamy.com/files/mmod_human_face_detector.dat [following]
--2022-05-17 16:36:35--  https://www.arunponnusamy.com/files/mmod_human_face_detector.dat
Resolving www.arunponnusamy.com (www.arunponnusamy.com)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to www.arunponnusamy.com (www.arunponnusamy.com)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 729940 (713K) [application/octet-stream]
Saving to: ‘mmod_human_face_detector.dat’

mmod_human_face_det 100%[===================>] 712.83K  --.-KB/s    in 0.06s   

2022-05-17 16:36:35 (12.1 MB/s) - ‘mmod_human_face_dete

In [ ]:
!bzip2 -d  '/content/shape_predictor_68_face_landmarks.dat.bz2'

bzip2: Output file /content/shape_predictor_68_face_landmarks.dat already exists.


In [ ]:
drive.mount('/content/gdrive')
!cp -r '/content/gdrive/My Drive/video-stable/model'/content
!cp -r '/content/gdrive/My Drive/video-stable/videos'/content

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
cp: cannot stat '/content/gdrive/My Drive/video-stable/model': No such file or directory
cp: cannot stat '/content/gdrive/My Drive/video-stable/videos': No such file or directory


In [ ]:
def save_image(path,file_name,images):
  for i in range(0,len(images)):
    # image name/id
    temp = file_name+'_'+str(i)+'.png'
    fn = os.path.join(path) + os.path.join(temp)
    cv2.imwrite(fn, images[i])


In [ ]:
cap=cv2.VideoCapture('/content/gdrive/MyDrive/video-stable/videos/video_data.mp4')
image_frame = []

while(cap.isOpened()):
    pic, frame = cap.read()
    if frame is None:
      break
    image_frame.append(frame)

cap.release()
plt.imshow(image_frame[0][:,:,::-1])

In [ ]:
from pandas.core import frame
# start capture video
capture = cv2.VideoCapture(0)

hog_face_detector = dlib.get_frontal_face_detector()

dlib_face_landmark = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat")

# loop over for reading webcam imags
while True:
  # _, frame = capture.read()
  frame = cv2.imread()
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
# get all the faces 
  faces = hog_face_detector(gray)
# loop allover the faces in the webcam
  for face in faces:
      face_landmarks = dlib_face_landmark(gray , face)
      # get 68 landmarks points x & y 
      for i in range(0,68):

        x = face_landmarks.part(i).x
        y = face_landmarks.part(i).y
        cv2.circle(frame, (x, y), 1, (0, 255, 255), 1)

  cv2.imshow("Face Landmarks", frame)
  key = cv2.waitKey(1)
  if key == 27:
        break
capture.release()
cv2.destroyAllWindows()


TypeError: ignored

In [ ]:
f_cascade = cv2.CascadeClassifier('lbpcascade_fronta lface.xml')
gray = cv2.cvtColor()